In [1]:
import sys
import gc
#sys.path
sys.path.insert(0, '../')

In [11]:
#sys.path

['../',
 'C:\\Users\\user\\Documents\\Accelerate\\CollaborativeProject\\recommendation_system\\collaborative_filtering',
 'C:\\Users\\user\\Anaconda3\\envs\\recommender\\python37.zip',
 'C:\\Users\\user\\Anaconda3\\envs\\recommender\\DLLs',
 'C:\\Users\\user\\Anaconda3\\envs\\recommender\\lib',
 'C:\\Users\\user\\Anaconda3\\envs\\recommender',
 '',
 'C:\\Users\\user\\Anaconda3\\envs\\recommender\\lib\\site-packages',
 'C:\\Users\\user\\Anaconda3\\envs\\recommender\\lib\\site-packages\\win32',
 'C:\\Users\\user\\Anaconda3\\envs\\recommender\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\user\\Anaconda3\\envs\\recommender\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\user\\Anaconda3\\envs\\recommender\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\user\\.ipython']

In [33]:
import pandas as pd
from Data_cleaning import get_clean_data
from Data_cleaning import get_merged_data_frame

### Load Data  
we now load the data using Alfred's framework.

In [34]:
df_merged = get_merged_data_frame(user_argv=10, isbn_argv=10, path='../data/')

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\Users\user\Anaconda3\envs\recommender\lib\site-packages\IPython\core\in

Actually we only need the data from df_ratings. But we merge the data and just deleting most of columns again.
We now end with less rows, because we have some ratings without corresponding books in df_books.

In [35]:
# df_merged.head()

In [36]:
df_books = df_merged[['isbn', 'title', 'author']]
df_books.set_index('isbn', inplace=True)
df_books.head()
#df_books.index

,title,author
isbn,,
034545104X,Flesh Tones: A Novel,M. J. Rose
034545104X,Flesh Tones: A Novel,M. J. Rose
034545104X,Flesh Tones: A Novel,M. J. Rose
034545104X,Flesh Tones: A Novel,M. J. Rose
034545104X,Flesh Tones: A Novel,M. J. Rose


In [37]:
print(df_merged.shape)
df_merged = df_merged.drop(['location', 'age', 'country', 'province',
                   'title', 'author', 'pub_year', 'publisher',
                   'url_s', 'url_m', 'url_l'], axis=1)
print(df_merged.shape)



(379317, 14)
(379317, 3)


In [38]:
gc.collect()

56

In [39]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('userItemRatingMatrix', 798352307),
 ('df_books', 84637260),
 ('data', 55236156),
 ('df_merged', 55236156),
 ('allUsers', 404411),
 ('distance', 136),
 ('get_clean_data', 136),
 ('get_merged_data_frame', 136),
 ('hamming', 136),
 ('nearestNeighbors', 136),
 ('np', 80),
 ('pd', 80)]

In [40]:
data = df_merged
data.head()
data[(data.user == '2313') & (data.rating == 5)]

,user,isbn,rating
1,2313,034545104X,5
51202,2313,0385482388,5
129253,2313,0399146431,5


To process 

In [41]:
data.shape

(379317, 3)

In [42]:
#%% CREATE RATINGS MATRIX
userItemRatingMatrix=pd.pivot_table(data, values='rating',
                                    index=['user'], columns=['isbn'])

In [25]:
#type(userItemRatingMatrix)

#userItemRatingMatrix.head()
#userItemRatingMatrix.columns
#2313
#userItemRatingMatrix.iloc[2313]
sum(userItemRatingMatrix.iloc[2313] == 5)

5

In [43]:
import numpy as np
from scipy.spatial.distance import hamming

def distance(user1,user2):
        try:
            user1Ratings = userItemRatingMatrix.transpose()[str(user1)]
            print(len(user1Ratings))
            user2Ratings = userItemRatingMatrix.transpose()[str(user2)]
            distance = hamming(user1Ratings,user2Ratings)
        except:
            distance = np.NaN
        return distance

def nearestNeighbors(user,K=10):
    allUsers = pd.DataFrame(userItemRatingMatrix.index)
    allUsers = allUsers[allUsers.user!=user]
    allUsers["distance"] = allUsers["user"].apply(lambda x: distance(user,x))
    KnearestUsers = allUsers.sort_values(["distance"],ascending=True)["user"][:K]
    return KnearestUsers

distance function

In [32]:
userItemRatingMatrix.shape

(6457, 15441)

In [47]:
print(distance('100004', '100009'))

user1Ratings = userItemRatingMatrix.transpose()[str(100004)]
user2Ratings = userItemRatingMatrix.transpose()[str(100009)]

hamming([1, 0, 0], [0, 1, 0])
#hamming([NaN, 0, 0], [NaN, 1, 0])
type(user1Ratings)

15441
1.0


pandas.core.series.Series

In [31]:
allUsers = pd.DataFrame(userItemRatingMatrix.index)

# allUsers = allUsers[allUsers.user!=user]
# allUsers["distance"] = allUsers["user"].apply(lambda x: distance(user,x))
# KnearestUsers = allUsers.sort_values(["distance"],ascending=True)["user"][:K]


# nearestNeighbors(100004)

allUsers.shape

(6457, 1)

In [50]:
df = userItemRatingMatrix.loc[['100004', '100009']]
#df[df.columns[df.apply(lambda s: len(s.unique()) == 1)]]
df[df.columns[df.apply(lambda s: sum(s.isnull()) == 1)]]

isbn,0060392452,0060502258,0060977337,0061015725,0312289871,0312981589,0312982518,0312983654,0345339703,0345339711,...,0553560441,0553582747,059035342X,0671701231,0671742760,0684853949,0743412273,0812543262,1551668246,1853260975
user,,,,,,,,,,,,,,,,,,,,,
100004,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,10.0,0.0,...,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100009,8.0,6.0,9.0,NaN,0.0,6.0,8.0,9.0,NaN,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,7.0,9.0


isbn,0060392452,0060502258,0060977337,0061015725,0312289871,0312981589,0312982518,0312983654,0345339703,0345339711,...,0553560441,0553582747,059035342X,0671701231,0671742760,0684853949,0743412273,0812543262,1551668246,1853260975
user,,,,,,,,,,,,,,,,,,,,,
100004,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,10.0,0.0,...,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100009,8.0,6.0,9.0,NaN,0.0,6.0,8.0,9.0,NaN,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,7.0,9.0


In [64]:
sum(df['0060502258'].isnull())

1

In [48]:
#distance()
type(userItemRatingMatrix)


userItemRatingMatrix.index
userItemRatingMatrix.head()


isbn,0002005018,0002251760,0002259834,0002558122,0006480764,000648302X,0006485200,000649840X,000651202X,0006512062,...,8845906884,8845915611,8878188212,8885989403,9074336329,9074336469,950491036X,9681500830,9681500954,9871138016
user,,,,,,,,,,,,,,,,,,,,,
100004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


???

In [82]:
# #%% DEBUGGING
# """NNRatings = userItemRatingMatrix[userItemRatingMatrix.index.isin(KnearestUsers)]
# NNRatings"""
# """avgRating = NNRatings.apply(np.nanmean).dropna()
# avgRating.head()"""
# """booksAlreadyRead = userItemRatingMatrix.transpose()[str(user)].dropna().index
# booksAlreadyRead"""
# """"avgRating = avgRating[~avgRating.index.isin(booksAlreadyRead)]"""


'"avgRating = avgRating[~avgRating.index.isin(booksAlreadyRead)]'

In [25]:
def bookMeta(isbn):
#df_books.set_index('isbn', inplace=True)
#    title = books.at[isbn,"title"]
#    author = books.at[isbn,"author"]
    title = df_books.at[isbn,"title"]
    author = df_books.at[isbn,"author"]
    return title, author

def faveBooks(user,N):
    userRatings = data[data["user"]==user]
    sortedRatings = pd.DataFrame.sort_values(userRatings,['rating'],ascending=[0])[:N] 
    sortedRatings["title"] = sortedRatings["isbn"].apply(bookMeta)
    return sortedRatings

def topN(user,N=3):
    KnearestUsers = nearestNeighbors(user)
    NNRatings = userItemRatingMatrix[userItemRatingMatrix.index.isin(KnearestUsers)]
    avgRating = NNRatings.apply(np.nanmean).dropna()
    booksAlreadyRead = userItemRatingMatrix.transpose()[user].dropna().index
    avgRating = avgRating[~avgRating.index.isin(booksAlreadyRead)]
    topNISBNs = avgRating.sort_values(ascending=False).index[:N]
    return pd.Series(topNISBNs).apply(bookMeta)


In [97]:
#%% DEBUGGING
"""N=3
topNISBNs = avgRating.sort_values(ascending=False).index[:N]
pd.Series(topNISBNs).apply(bookMeta)"""
"""user = '204622'
topN(user)"""


"user = '204622'\ntopN(user)"

In [27]:
topNISBNs = avgRating.sort_values(ascending=False).index[:3]

NameError: name 'avgRating' is not defined

In [98]:
#%% DEBUGGING
"""N=3
topNISBNs = avgRating.sort_values(ascending=False).index[:N]
pd.Series(topNISBNs).apply(bookMeta)"""
"""user = '204622'
topN(user)"""


"user = '204622'\ntopN(user)"

In [100]:
bookMeta('034545104X')

('Flesh Tones: A Novel', 'M. J. Rose')

In [102]:
faveBooks('204622', 5)

,user,isbn,rating,title
8575,204622,0385504209,10,"(The Da Vinci Code, Dan Brown)"
65948,204622,0671027360,10,"(Angels &amp; Demons, Dan Brown)"
169393,204622,0060935464,9,"(To Kill a Mockingbird, Harper Lee)"
499333,204622,1878424114,9,(The Seven Spiritual Laws of Success: A Practi...
484364,204622,0671666258,8,"(American Star, Jackie Collins)"


In [103]:
topN('204622', 5)

C:\Users\user\Anaconda3\envs\recommender\lib\site-packages\pandas\core\apply.py:242: RuntimeWarning: Mean of empty slice
  labels=labels)


0              (Love, Greg &amp; Lauren, Greg Manning)
1    (The Two Towers (The Lord of the Rings, Part 2...
2    (Harry Potter and the Sorcerer's Stone (Book 1...
3                       (Charlotte's Web, E. B. White)
4             (The Secret Life of Bees, Sue Monk Kidd)
Name: isbn, dtype: object

In [ ]:
**what the hell is this?**

In [41]:
# #%% THRESHOLD CI
# """"""from scipy.stats import sem, t
# from scipy import mean
# confidence = 0.95
# data = ratings_per_isbn['count']

# n = len(data)
# m = mean(datca)
# std_err = sem(data)
# h = std_err * t.ppf((1 + confidence) / 2, n - 1)

# start = m - h
# print (start)"""
# #%% VIS ISBN & USER COUNT
# """import seaborn as sns
# ax = sns.distplot(ratings_per_isbn['count'])
# ax2 = ax.twinx()
# sns.boxplot(x=ratings_per_isbn['count'], ax=ax2)
# ax2.set(ylim=(-0.5, 10))"""
